# Movie Ratings Capstome

#### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')

### Acquire the Data from  https://www.imdb.com/interfaces/

In [9]:
# (def get_data(use_cache = True):
    
#     titles_ratings = 'movie_ratings.csv'
#     name_basic = 'name_basic.csv'
#     titles_basics = 'titles_basics.csv'
#     titles_crew = 'titles_crew.csv'
#     titles_principals = 'titles_principals.csv'
#     titles_akas = 'titles_akas.csv'

#     if os.path.exists([['titles_ratings', 'name_basic',
#                        'titles_basics','titles_crew',
#                        'titles_principals', 'titles_akas']]) and use_cache:
#  print('Getting csv file..')
 
#  pd.read_csv()
         
# )

#### Initial read of the tsv.gz file

In [8]:
titles_ratings = pd.read_csv('title.ratings.tsv.gz', sep = '\t')
name_basic = pd.read_csv('name.basics.tsv.gz', sep = '\t')
titles_basics = pd.read_csv('title.basics.tsv.gz', sep = '\t')
titles_crew = pd.read_csv('title.crew.tsv.gz', sep = '\t')
titles_principals = pd.read_csv('title.principals.tsv.gz', sep = '\t')
titles_akas = pd.read_csv('title.akas.tsv.gz', sep = '\t')


# episode = pd.read_csv('title.episode.tsv.gz', sep = '\t')# Agreed to drop

#### Save to local csv file (add function to skip this step if already performed)

In [7]:
# titles_ratings.to_csv('titles_ratings.csv', index=False)
# name_basic.to_csv('name_basic.csv', index=False)
# titles_basics.to_csv('titles_basics.csv', index=False)
# titles_crew.to_csv('titles_crew.csv', index=False)
# titles_principals.to_csv('titles_principals.csv', index=False)
# titles_akas.to_csv('titles_akas.csv', index=False)

#### Read locally cached files

In [9]:
titles_ratings = pd.read_csv('titles_ratings.csv')
name_basic = pd.read_csv('name_basic.csv')
titles_basics = pd.read_csv('titles_basics.csv')
titles_crew = pd.read_csv('titles_crew.csv')
titles_principals = pd.read_csv('titles_principals.csv')
titles_akas = pd.read_csv('titles_akas.csv')

In [12]:
titles_ratings.head(1)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1882


In [13]:
titles_ratings.shape

(1247233, 3)

In [14]:
name_basic.head(1)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0053137,tt0072308,tt0050419"


In [15]:
name_basic.shape

(11654331, 6)

In [16]:
titles_basics.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"


In [17]:
titles_basics.shape

(8946058, 9)

In [19]:
titles_crew.head(1)

,tconst,directors,writers
0,tt0000001,nm0005690,\N


In [20]:
titles_crew.shape

(8946058, 3)

In [21]:
titles_principals.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"


In [22]:
titles_principals.shape

(50453291, 6)

In [33]:
titles_akas.head(1)

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0


In [24]:
titles_akas.shape

(32079668, 8)

### Join the tables

- join='outer' - 0 information loss, all union

- join='inner' - intersection 

In [25]:
# Rename columns in akas set

titles_akas = titles_akas.rename(columns = {'titleId': 'tconst'})

In [31]:
# Reset index on titles_basics

titles_basics = titles_basics.reset_index(drop = True) 

In [38]:
titles_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


#### Concat titles_basic with titles_akas

In [46]:
akas_titles_basics = pd.merge(titles_basics,titles_akas, on = 'tconst' , sort=False)

In [48]:
akas_titles_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32074449,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family",5,Episódio #3.20,PT,pt,\N,\N,0
32074450,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family",6,Episodio #3.20,IT,it,\N,\N,0
32074451,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family",7,एपिसोड #3.20,IN,hi,\N,\N,0
32074452,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short,1,The Wind,DE,\N,imdbDisplay,\N,0


In [ ]:
# Join all tables 

(titles_basics.set_index('tconst')\
            .join(titles_akas.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
            .join(titles_crew.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
            .join(titles_principals.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
            .join(titles_ratings.set_index('tconst'),how ='left', on = 'tconst', rsuffix = '2')
)